In [ ]:
############################################
#PARA GENERAR SUS KEYS, DEBEN DE REGISTRARSE EN (https://alpaca.markets/) SE REGISTRAN GRATIS,
#CONFIRMAN SU CORREO, Y CUANDO ENTRAN A SU PERFIL LES PEDIRA AUTENTIFICARSE CON QR
#YO USE GOOGLE AUTHENTICADOR PARA ESCANEAR EL QR Y OBTENER EL CODIGO DE SEGURIDAD
#YA INGRESADO EL CODIGO SE VAN A HOME Y EN EL LADO DERECHO UN POCO MAS ABAJO VERAN UNA SECCION API KEYS LE DAN
# A GENERAR Y SE LES MOSTRARA 2 KEYS ESAS LAS REEMPLAZAN EN LA SECCION 2 DE LA SIGUIENTE CELDA
##########################################

In [ ]:
# 1. Instalación e Importación
import pandas as pd
from datetime import datetime, timedelta
import os
# Importaciones necesarias de la librería alpaca-py
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient

In [ ]:
# =================================================================
# 2. CONFIGURACIÓN DE ACCESO (¡REEMPLAZA ESTO CON TUS CLAVES!)
# =================================================================
# --- Obtén estas claves de tu Dashboard de Paper Trading en Alpaca ---
API_KEY_ID = 'PKZD4OTDXPKCPYJQI42YRONFXS'
SECRET_KEY = 'B8vipqTSDWvnvc3XRdUjQqyADCN4iR2GsMDtwYAniBXV'
# =================================================================

# 3. Parámetros de Extracción
SYMBOL = "SPY"
INTERVAL = TimeFrame.Minute # Granularidad de 1 minuto ($1m$)
# Definimos el rango: Hoy hasta hace 1 año (aproximadamente 365 días)
END_TIME = datetime.now()
START_TIME = END_TIME - timedelta(days=365)
NOMBRE_ARCHIVO_CSV = f"{SYMBOL}_1min_1year_alpaca_BATS.csv"

print(f"--- Iniciando la extracción de datos: {SYMBOL} a 1 minuto por 1 año (Feed BATS) ---")

# =================================================================
# 4. Descarga de Datos
# =================================================================

spy_data = pd.DataFrame() # Inicializamos el DataFrame
try:
    # 4.1 Inicializar el Cliente de Datos Históricos
    client = StockHistoricalDataClient(API_KEY_ID, SECRET_KEY)

    # 4.2 Definir los Parámetros de la Solicitud
    request_params = StockBarsRequest(
        symbol_or_symbols=[SYMBOL],
        timeframe=INTERVAL,
        start=START_TIME,
        end=END_TIME,
        # CORRECCIÓN CLAVE: Usamos 'iex' en lugar del obsoleto 'BATS'
        feed='iex'
    )


    # 4.3 Descargar los Datos (Puede tomar tiempo debido al volumen de datos)
    bars = client.get_stock_bars(request_params)

    # 4.4 Convertir el objeto Bars a un DataFrame de Pandas
    spy_data = bars.df

except Exception as e:
    print(f"ERROR durante la descarga. Mensaje: {e}")

# =================================================================
# 5. Guardar y Confirmar
# =================================================================

if spy_data.empty:
    print("\nAdvertencia: El DataFrame está vacío. La descarga falló o no se encontraron datos en el rango.")
else:
    # Limpieza y Reestructuración: Eliminamos el multi-índice 'symbol'
    spy_data = spy_data.reset_index(level='symbol', drop=True)
    spy_data = spy_data.sort_index() # Aseguramos el orden cronológico

    # Guardar en CSV
    spy_data.to_csv(NOMBRE_ARCHIVO_CSV)

    # Muestra información de confirmación
    print("\nExtracción y guardado completados.")
    print(f"Número total de registros (barras de 1m): {len(spy_data)}")
    print(f"Rango de Fechas: {spy_data.index.min()} a {spy_data.index.max()}")
    print(f"Datos guardados exitosamente en: {os.path.abspath(NOMBRE_ARCHIVO_CSV)}")

    print("\nPrimeras 5 barras (1m):")
    print(spy_data.head())
    print("\nÚltimas 5 barras (1m):")
    print(spy_data.tail())

--- Iniciando la extracción de datos: SPY a 1 minuto por 1 año (Feed BATS) ---

✅ Extracción y guardado completados.
Número total de registros (barras de 1m): 96284
Rango de Fechas: 2024-11-18 14:30:00+00:00 a 2025-11-17 16:11:00+00:00
Datos guardados exitosamente en: C:\UCM\MachineLearning\SPY_1min_1year_alpaca_BATS.csv

Primeras 5 barras (1m):
                              open    high      low    close  volume  \
timestamp                                                              
2024-11-18 14:30:00+00:00  585.950  586.04  585.880  585.890  1463.0   
2024-11-18 14:31:00+00:00  585.690  585.69  585.540  585.590  2461.0   
2024-11-18 14:32:00+00:00  585.630  585.63  585.445  585.445  3615.0   
2024-11-18 14:33:00+00:00  585.370  585.61  585.370  585.520  2194.0   
2024-11-18 14:34:00+00:00  585.525  585.61  585.525  585.550  2575.0   

                           trade_count        vwap  
timestamp                                           
2024-11-18 14:30:00+00:00         55.0  5